In [2]:
%cd ..

/home/jbananafish/Desktop/Master/Thesis/code/gcnboost


In [30]:
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import GCNConv, SAGEConv, Linear, GraphConv, GATConv, to_hetero
import torch_geometric.transforms as T

from src.data.artgraph import ArtGraph

In [4]:
torch.manual_seed(1)
torch.cuda.manual_seed(1)

In [5]:
#base_data = ArtGraph("./ekg", preprocess='node2vec', features=True, type='ekg')
base_data = ArtGraph("data", preprocess='node2vec', transform=T.ToUndirected(), features=True, type='ekg')

In [19]:
data = base_data[0]

## Some graph-level information

In [5]:
print(f"Number of artist classes: {base_data.num_classes['artist']}")
print(f"Number of style classes: {base_data.num_classes['style']}")
print(f"Number of genre classes: {base_data.num_classes['genre']}")
print(f"Number of input features: {base_data.num_features}")

Number of artist classes: 300
Number of style classes: 83
Number of genre classes: 50
Number of input features: 128


## Some node-level information

In [6]:
data = base_data[0]

In [13]:
print(data)

HeteroData(
  artwork={
    x=[61477, 128],
    y_artist=[61477],
    y_style=[61477],
    y_genre=[61477],
    train_mask=[61477],
    val_mask=[61477],
    test_mask=[61477]
  },
  artist={ x=[300, 128] },
  gallery={ x=[1090, 128] },
  city={ x=[665, 128] },
  country={ x=[64, 128] },
  style={ x=[83, 128] },
  period={ x=[53, 128] },
  genre={ x=[50, 128] },
  serie={ x=[610, 128] },
  auction={ x=[5, 128] },
  tag={ x=[5146, 128] },
  media={ x=[160, 128] },
  subject={ x=[2161, 128] },
  training_node={ x=[108, 128] },
  field={ x=[65, 128] },
  movement={ x=[121, 128] },
  people={ x=[48, 128] },
  (artist, influenced_rel, artist)={ edge_index=[2, 62] },
  (artist, subject_rel, subject)={ edge_index=[2, 3648] },
  (artist, training_rel, training_node)={ edge_index=[2, 130] },
  (artist, field_rel, field)={ edge_index=[2, 323] },
  (artist, movement_rel, movement)={ edge_index=[2, 286] },
  (artist, patrons_rel, people)={ edge_index=[2, 45] },
  (artist, teacher_rel, artist)={ ed

In [31]:
class HomoGNN(torch.nn.Module):
    def __init__(self, operator=GCNConv, input_channels=128, hidden_channels=16, out_channels=300, num_layers=1, dropout=0.5, skip=False):
        super(HomoGNN, self).__init__()
        self.dropout = dropout
        self.skip = skip
        self.convs = torch.nn.ModuleList()
        self.lins = torch.nn.ModuleList()
        #self.convs.append(operator(input_channels, hidden_channels))
        for _ in range(num_layers):
            conv = operator(-1, hidden_channels)
            lin = Linear(-1, hidden_channels)
            self.convs.append(conv)
            self.lins.append(lin)
        self.conv_out = operator(hidden_channels, out_channels)
        
    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            if self.skip:
                x = conv(x, edge_index).relu() + self.lins[i](x)
            else:
                x = conv(x, edge_index).relu()
            x = F.dropout(x, self.dropout)
        x = self.conv_out(x, edge_index)
        return F.log_softmax(x, dim=1)

In [32]:
class HomoSGNN(torch.nn.Module):
    def __init__(self, operator, input_channels, hidden_channels, out_channels, n_layers, dropout, skip):
        super(HomoSGNN, self).__init__()
        self.gnn = HomoGNN(operator, input_channels, hidden_channels, out_channels, n_layers, dropout, skip)

    def forward(self, x, edge_index):
        return [self.gnn(x, edge_index)]

class HomoMGNN(torch.nn.Module):
    def __init__(self, operator, input_channels, hidden_channels, out_channels, n_layers, dropout, skip):
        super(HomoMGNN, self).__init__()
        self.gnn_artist = HomoGNN(operator, input_channels, hidden_channels, out_channels['artist'], n_layers, dropout, skip)
        self.gnn_style = HomoGNN(operator, input_channels, hidden_channels, out_channels['style'], n_layers, dropout, skip)
        self.gnn_genre = HomoGNN(operator, input_channels, hidden_channels, out_channels['genre'], n_layers, dropout, skip)

    def forward(self, x, edge_index):
        return [self.gnn_artist(x, edge_index), self.gnn_style(x, edge_index), self.gnn_genre(x, edge_index)]
